# 3. 신경망
* * *

+ 퍼셉트론으로 복잡한 함수도 표현할 수 있음
+ 그러나, 가중치를 설정하는 작업 ( 원하는 결과를 출력하도록 가중치 값을 적절히 정하는 작업 ) 은 여전히 사람이 수동으로 해야함
</br></br>
+ 신경망의 중요한 성질 == 가중치 매개변수의 적절한 값을 데이터로부터 잗ㅇ으로 학습하는 능력

## 3.6 손글씨 숫자 인식
+ 이미 학습된 매개변수를 사용하여 학습 과정은 생략하고, 추론 과정만 구현
+ 추론과정 == 신경망의 순전파 ( forward propagation )
</br></br>
+ 신경망은 두 단계를 거쳐 문제를 해결함
    + 훈련 데이터 ( 학습 데이터 ) 를 사용해 가중치 매개변수 학습
    + 추론 단계에서 앞서 학습한 매개변수를 사용하여 입력 데이터를 분류

##### <br><br><span style="color: mediumaquamarine;">**MNIST 데이터셋**</spen>

![](../img/mnist.png)

+ <spen style="color: gold;">**MNIST 데이터셋**</spen> == 손글씨 숫자 이미지 집합 ( 0 ~ 9 )
    + 기계학습 분야에서 아주 유명한 데이터셋
    + 간단한 실험부터 논문으로 발표되는 연구까지 다양한 곳에서 이용함
    + 이미지 인식이나 기계학습 논문들을 읽다 보면 실험용 데이터로 자주 등장하는 것을 확인할 수 있음
+ 일반적으로 훈련 이미지들을 사용하여 모델을 학습하고, 학습한 모델로 시험 이미지들을 얼마나 정확하게 분류하는지를 평가함
</br></br>
+ 28 X 28 크기의 회색조 이미지 ( 채널 1 )
+ 각 픽셀은 0 ~ 255까지의 값을 취함
+ 가 이미지가 실제 의미하는 숫자가 레이블로 붙어있음

In [1]:
import sys, os
sys.path.append(os.pardir) # 부모 디렉터리의 파일을 가져올 수 있도록 설정
from dataset.mnist import load_mnist

# load_mnist가 MNIST 데이터를 받아와야 하니 최초 실행 시에는 인터넷에 연결된 상태여야 한다.
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

# 각 데이터의 형상 출력
print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


+ ```load_mnist``` 함수 == 읽은 MNIST 데이터를 "( 훈련 이미지, 훈련 레이블 ), ( 시험 이미지, 시험 레이블 )" 형식으로 반환
    <spen style="color: rosybrown;">**인수**</spen>
    + ```normalize``` == 입력 이미지의 픽셀 값을 0.0 ~ 1.0 사이의 값으로 정규화할지를 정함
        + False == 입력 이미지의 픽셀은 원래 값 그대로 0 ~ 255 사이의 값을 유지
    + ```flatten``` == 입력 이미지를 평탄하게, 즉 1차원 배열로 만들지를 정함
        + False == 입력 이미지가 1 X 28 X 28의 3차원 배열
        + True == 입력 이미지가 784개의 원소로 이뤄진 1차원 배열
    + ```one_hot_lable``` == 원-핫 인코딩 형태로 저장할지를 정함
        + False == '7', '2'와 같이 숫자 형태의 레이블을 저장
        + True == 레이블을 원-핫 인코딩하여 저장 ( <spen style="color: gold;">**원-핫 인코딩 ( one-hot encoding )**</spen> == 정답을 뜻하는 원소만 1이고 ( hot하고 ) 나머지는 모두 0인 배열 )

+ <spen style="color: gold;">**pickle ( 피클 )**</spen> == 프로그램 실행 중에 특정 객체를 파일로 저장하는 기능
    + 저장해둔 pickle 파일을 로드하면 실행 당시의 객체를 즉시 복원할 수 있음
+ PIL ( Python Image Library ) 모듈 == 이미지 표시에 사용

In [2]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from PIL import Image

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

img = x_train[0]
label = t_train[0]
print(label)

print(img.shape)
img = img.reshape(28, 28) # 원래 이미지의 모양으로 변형
print(img.shape)

img_show(img)

5
(784,)
(28, 28)


<spen style="color: palevioletred;">**주의사항**</spen>
+ ```flatten=True```로 설정해 읽어들인 이미지는 1차원 넘파이 배열로 저장되어있음
    + 이미지를 표시할 때는 원래 형상인 28 X 28 크기로 다시 변형
+ ```reshape()``` 메서드에 원하는 형상을 인수로 지정하면 넘파이 배열의 형상을 바꿀 수 있음
+ ```Image.fromarray()``` == 넘파이로 저장된 이미지 데이터를 PIL용 데이터 객체로 변환

##### <br><br><span style="color: mediumaquamarine;">**신경망의 추론 처리**</spen>
+ MNIST 데이터셋으로 추론을 수행하는 신경망 구현
    + <spen style="color: rosybrown;">**입력층 뉴런**</spen> == 784개 ( 이미지 크기 28 X 28 = 784 )
    + <spen style="color: rosybrown;">**출력층 뉴런**</spen> == 10개 ( 0 ~ 9까지의 숫자를 구분하는 문제이기 때문 ← 총 10개 )  
    + <spen style="color: rosybrown;">**은닉층 2개**</spen> ( 뉴런 수 임의로 정함 )
        + 첫번째 은닉층 == 50개
        + 두번째 은닉층 == 100개

In [3]:
import pickle
from fucntion import sigmoid, softmax

def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test

def init_network():
    with open("../dataset/sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)

    return y

+ ```init_network()``` == pickle 파일인 sample_weight.pkl에 저장된 '학습된 가중치 매개변수'를 읽음
    + 가중치와 편향 매개변수가 딕셔너리 변수로 저장되어 있음

In [4]:
x, t = get_data()
network = init_network()

accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y) # 확률이 가장 높은 원소의 인덱스를 얻는다.
    if p == t[i]:
        accuracy_cnt += 1

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352


+ **신경망에 의한 추론을 수행해보고, <spen style="color: palevioletred;">정확도 ( accuracy )</span>** **를 평가**
    1. MNIST 데이터셋을 얻고 네트워크를 생성
    2. for문을 돌며 x에 저장된 이미지 데이터를 하나씩 꺼내 ```predict()``` 함수로 분류
        + ```predict()``` 함수 == 각 레이블의 확률을 넘파이 배열로 반환
    3. ```np.argmax()``` 함수로 배열 중 값이 가장 큰 ( 확률이 가장 높은 ) 원소의 인덱스를 구함 ← 예측결과
    4. 신경망이 예측한 답변과 정답 레이블을 비교하여 맞힌 숫자 ( accuracy_cnt ) 를 세고, 이를 전체 이미지 숫자로 나누어 정확도를 구함
+ 입력 이미지 데이터에 대한 전처리 작업으로 정규화를 수행함
+ 올바르게 분류한 비율 == 93.52%

+ <spen style="color: gold;">**정규화 ( normalization )**</spen> == 데이터를 특정 범위로 변환하는 처리
+ <spen style="color: gold;">**전처리 ( pre-processing )**</spen> == 신경망의 입력 데이터에 특정 변환을 가하는 것
    + 전처리를 통해 식별 능력을 개선하고 학습 속도를 높이는 등의 사례가 많이 제시되고있음
    + 현업에서는 데이터 전체의 분포를 고려해 전처리하는 경우가 많음
        + 데이터 전체 평균과 표준편차를 이용하여 데이터들이 0을 중심으로 분포하도록 이동시킴
        + 데이터의 확산 범위를 제한하는 정규화를 수행
+ <spen style="color: gold;">**데이터 백색화 ( whitedning )**</spen> == 전체 데이터를 균일하게 분포시킴

##### <br><br><span style="color: mediumaquamarine;">**배치 처리**</spen>
+ <spen style="color: gold;">**배치 ( batch )**</spen> == 묶음 == 하나로 묶은 입력 데이터 ( 일괄적으로 모아서 한번에 처리하는 작업 )
    + 이점 == 이미지 1 장당 처리 시간을 대폭 줄여줌
        + 수치 계산 라이브러리 대부분이 큰 배열을 효율적으로 처리할 수 있도록 고도로 최적화되어있음
        + 커다란 신경망에서는 데이터 전송이 병목으로 작용하는 경우가 자주 있는데, 배치 처리를 함으로써 버스에 주는 부하를 줄임
            + <spen style="color: palevioletred;">**느린 I/O를 통해 데이터를 읽는 횟수가 줄어 빠른 CPU나 GPU로 순수 게산을 수행하는 비율이 높아짐**</spen>
            + <spen style="color: palevioletred;">**GPU와 CPU 간에 데이터를 주고받을때 데이터 전송 속도가 계산 속도보다 느리면 병목이 생김**</spen>
+ 입력 데이터와 가중치 매개변수의 '형상'에 주의해서 구현

In [5]:
x, _ = get_data()
network = init_network()
W1, W2, W3 = network['W1'], network['W2'], network['W3']

print(x.shape)
print(x[0].shape)
print("W1 = ", W1.shape)
print("W2 = ", W2.shape)
print("W3 = ", W3.shape)

(10000, 784)
(784,)
W1 =  (784, 50)
W2 =  (50, 100)
W3 =  (100, 10)


신경망 각 층의 배열 형상의 추이  
<img src="../img/neural_network_array1.png" width='800'>

+ 다차원 배열의 대응하는 차원의 원소 수가 일치함
+ <spen style="color: palevioletred;">**최종 결과**</spen> == 원소가 10개인 1차원 배열 $y$가 출력됨
    + 이미지 1장만 입력했을 때의 처리 흐름 == 원소 784개로 구성된 1차원 배열이 입력되어 마지막에는 원소 10개인 1차원 배열이 출력되는 흐름

배치 처리를 위한 배열들의 형상 추이  
<img src="../img/neural_network_array2.png" width='800'>

+ 이미지 여러장을 한꺼번에 넘기는 경우 == 묶어서 predict() 함수에 한번에 넘김
    + ex ) x의 형상을 100 X 784로 바꿔서 100장 분량의 데이터를 하나의 입력 데이터로 표현
        + 입력 데이터의 형상 == 100 X 784
        + 출력 데이터의 형상 == 100 X 10
        + 100장 분량의 입력 데이터의 결과가 한 번에 출력됨 ( x[0], y[0] == 0번째 이미지와 추론 결과, x[1], y[1] == 1번째 이미지와 추론 결과 )

In [6]:
x, t = get_data()
network = init_network()

batch_size = 100 # 배치 크기
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size] # 100장씩 묶음
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])

print("Accuracy :" + str(float(accuracy_cnt) / len(x))) # 정확도

Accuracy :0.9352


```range()``` 함수
+ ```range(start, end)``` ( 인수 2개 ) == ```start``` ~ ```end - 1``` 까지의 정수를 차례로 반환하는 반복자를 돌려줌
+ ```range( start, end, step )``` ( 인수 3개 ) == ```start``` ~ ```end - 1``` 까지 ```step``` 간격으로 증가하는 정수를 반환하는 반복자를 돌려줌
+ <spen style="color: gold;">**반복자 ( iterator )**</spen> == 프로그래밍에서 배열을 참조할 때 사용하는 개체 중 하나로, 배열을 순차적으로 접근하고 싶을 때 사용

In [7]:
print(list(range(0, 10)))
print(list(range(0, 10, 3)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 3, 6, 9]


+ ```range()``` 함수가 반환하는 반복자를 바탕으로 ```x[i:i+batch_size]```에서 입력 데이터를 묶음
+ ```x[i:i+batch_size]``` == 입력 데이터의 ```i```번째부터 ```i+batch_size```번째까지의 데이터를 묶음

차원 ( 방향 ) 을 지정하는 매개변수  
    + ```axis=0``` == 열 ( column ) 방향으로 연산 수행  
    + ```axis=1``` == 행 ( row ) 방향으로 연산 수행

In [8]:
x = np.array([[0.1, 0.8, 0.1], [0.3, 0.1, 0.6], [0.2, 0.5, 0.3], [0.8, 0.1, 0.1]])
y = np.argmax(x, axis=1)
print(y)

[1 2 1 0]


+ 배치 단위로 분류한 결과를 실제 답과 비교
    + ```==``` 연산자를 사용해 넘파이 배열끼리 비교하여 ```True / False```로 구성된 ```bool``` 배열 ( 결과 배열 ) 생성
    + 결과 배열에서 ```True```가 몇 개인지 셈

In [9]:
y = np.array([1, 2, 1, 0])
t = np.array([1, 2, 0, 0])
print(y == t)
print(np.sum(y == t))

[ True  True False  True]
3


<spen style="color: palevioletred;">**데이터를 배치로 처리함으로써 효율적이고 빠르게 처리할 수 있다.**</spen>